In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.array as da
from dask import dataframe as dd
import dask
import glob
from scipy import stats

In [ ]:
#Dont run the next cell!

In [ ]:
files = glob.glob('../instance_files/createdb_iceagg_rand*')
for file in files:
    print(file)
    df = pd.read_pickle(file)
    df = pd.DataFrame(df)
    df[['ncrystals', 'a', 'b', 'c', 'cplx', 'phi2D', 'mono_phi',
       'mono_r']].to_parquet('../instance_files/parquet_files/with_points/'+file.split('/')[-1],\
                                                              compression='gzip')
    print('done transfering %s to parquet' %file)

In [ ]:
%%time
ddf = dd.read_parquet("../instance_files/parquet_files/*rand*", engine="pyarrow").compute()

In [ ]:
%%time
ddf = dd.read_parquet("../instance_files/parquet_files/*flat*", engine="pyarrow").compute()

In [ ]:
ddf = pd.read_hdf('df_area_ratio_no_points.h5')

In [ ]:
f = open('../instance_files/instance_db_aggagg_rand', 'rb')
results = pickle.load(f)
agg_as_aggagg_rand, agg_bs_aggagg_rand, agg_cs_aggagg_rand, phi2Ds_aggagg_rand, cplxs_aggagg_rand, dds_aggagg_rand= \
                results['agg_as'], results['agg_bs'], results['agg_cs'], results['phi2Ds'], results['cplxs'], results['dds']

f.close()

In [ ]:
print(np.shape(cplxs_aggagg_rand))


In [ ]:
ddf['agg_phi'] = ddf['c']/ddf['a']
ddf['agg_r'] = np.power((np.power(ddf['a'], 2) * ddf['c']), (1./3.))
#ddf['agg_r'] = ddf['agg_r'][ddf.agg_r < 5000]
ddf = ddf[ddf.agg_r < 5000]
ddf.agg_phi.describe().round(2)

In [ ]:
#truncall = ddf
# trunc20 = ddf[(ddf['ncrystals'] <= 20)]
# trunc10 = ddf[(ddf['ncrystals'] <= 10)]
# trunc5 = ddf[(ddf['ncrystals'] <= 5)]
truncall = ddf[(ddf['mono_phi'] <= 50) & (ddf['mono_phi'] >= 0.6)]
trunc20 = ddf[(ddf['mono_phi'] <= 50) & (ddf['mono_phi'] >= 0.6) & (ddf['ncrystals'] <= 20)]
trunc10 = ddf[(ddf['mono_phi'] <= 50) & (ddf['mono_phi'] >= 0.6) & (ddf['ncrystals'] <= 10)]
trunc5 = ddf[(ddf['mono_phi'] <= 50) & (ddf['mono_phi'] >= 0.6) & (ddf['ncrystals'] <= 5)]
trunc2 = ddf[(ddf['mono_phi'] <= 50) & (ddf['mono_phi'] >= 0.6) & (ddf['ncrystals'] <= 2)]

In [ ]:
#read in CPI data 
#all campaings in one file
df_CPI = pd.read_csv('all_campaigns.csv')
#only use aggregates
df_CPI = df_CPI[(df_CPI['classification'] == 'agg')]

In [ ]:
df_CPI['IPAS_ellipse'].min()

In [ ]:
#gather columns that are the same as IPAS dataframe to merge
CPI =  df_CPI[['IPAS_ellipse','filled_circular_area_ratio', 'complexity']]
CPI.rename(columns={'IPAS_ellipse':'Aspect Ratio' ,"filled_circular_area_ratio": "Area Ratio", 'complexity':'Complexity'}, inplace=True)
ddf.rename(columns={"area ratio": "Area Ratio", 'cplx':'Complexity', 'phi2D': 'Aspect Ratio'}, inplace=True)


In [ ]:
#combine CPI and IPAS data to plot
cdf = pd.concat([CPI, ddf[['Complexity', 'Aspect Ratio', 'Area Ratio']]], keys=['CPI', 'IPAS'], names=["Source"]).reset_index().drop(columns='level_1')

In [ ]:
#for plotting. need a source column for hue argument in seaborn
cdf = pd.melt(cdf,id_vars=['Source'],value_vars=['Complexity', 'Aspect Ratio', 'Area Ratio'],
             value_name='Value')

In [ ]:
df_nonan = df_CPI[df_CPI['IPAS_ellipse'].notnull()]

In [ ]:
#bins, edges = np.histogram(np.isfinite(truncall['phi2D']), density=True, bins = 70)
n, bins, patchs = plt.hist(truncall['phi2D'][np.isfinite(truncall['phi2D'])], density=True, bins = 70);

In [ ]:
n, bins, patchs = plt.hist(truncall['area ratio'][np.isfinite(truncall['area ratio'])], density=True, bins = 70);

In [ ]:
n, bins, patchs = plt.hist(CPI['Area Ratio'], density=True, bins = 70);
plt.axvline(x=0.44, color='magenta', label='RMSE 1')

In [ ]:
len(cplxs_aggagg_rand[:,:,:].flatten())

In [ ]:
xlarge=24; large = 20; med = 16; small = 14
params = {'axes.titlesize': xlarge,
          'legend.fontsize': small,
          'figure.figsize': (7,7),
          'axes.labelsize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large,
          "font.family": "serif"}
import matplotlib as mpl  
mpl.rc('font',family='serif')
plt.rcParams.update(params)

In [ ]:
stats.mode(df_nonan['IPAS_ellipse'])

In [ ]:
stats.mode(trunc5['cplx'])

In [ ]:
#fig, ax = plt.subplots()
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,12))
sns.set_style("white")

g=sns.distplot(df_nonan['IPAS_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'#FA2400'}, label='CPI aggregates',ax=ax2)

# sns.distplot(df_nonan['phi_ellipse'], hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3,'color':'red'}, label='CPI (opencv ellipse)')

g=sns.distplot(truncall['phi2D'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ $\leq$ 30',ax=ax2)

g=sns.distplot(trunc20['phi2D'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'--','color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ $\leq$ 20',ax=ax2)

g=sns.distplot(trunc10['phi2D'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ $\leq$ 10',ax=ax2)

#g=sns.distplot(trunc5['phi2D'], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':':','color':'lightskyblue'}, label='IPAS $\phi_{2D}$ $n_m$ < 5',ax=ax2)

g=sns.distplot(truncall['agg_phi'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'#0B1CE0'}, label='IPAS $\phi_{ca}$ $n_m$ $\leq$ 30',ax=ax2)

g=sns.distplot(trunc20['agg_phi'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'--','color':'#0B1CE0'}, label='IPAS $\phi_{ca}$ $n_m$ $\leq$ 20',ax=ax2)

g=sns.distplot(trunc10['agg_phi'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'#0B1CE0'}, label='IPAS $\phi_{ca}$ $n_m$ $\leq$ 10',ax=ax2)

#g=sns.distplot(trunc5['agg_phi'], hist = False, kde = True, norm_hist=True,
#             kde_kws = {'linewidth': 3, 'linestyle':':','color':'#0B1CE0'}, label='IPAS $\phi_{ca}$ $n_m$ < 5',ax=ax2)
ax2.set_xlabel("Aspect Ratio",fontsize=18)
ax2.set_ylabel("Frequency",fontsize=18)
ax2.set_ylim(0, 5)
ax2.set_xlim(0.0, 1.0)
ax2.legend(loc='upper center', ncol=2)

sns.distplot(df_CPI['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI aggregates', ax=ax1)
sns.distplot(truncall['cplx'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'#0B1CE0'}, label='IPAS $n_m$ $\leq$ 30', ax=ax1)
sns.distplot(trunc20['cplx'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'--', 'color':'#0B1CE0'}, label='IPAS $n_m$ $\leq$ 20', ax=ax1)
sns.distplot(trunc10['cplx'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'-.', 'color':'#0B1CE0'}, label='IPAS $n_m$ $\leq$ 10', ax=ax1)
# sns.distplot(cplxs_aggagg_rand[:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
#              kde_kws = {'linewidth': 3, 'linestyle':'-.','color':'#0B1CE0'}, label='IPAS agg-agg', ax=ax1)

sns.distplot(trunc5['cplx'], hist = False, kde = True, norm_hist=True,
            kde_kws = {'linewidth': 3, 'linestyle':':', 'color':'#0B1CE0'}, label='IPAS $n_m$ < 5', ax=ax1)

ax1.set_xlabel("Complexity",fontsize=18)
ax1.set_ylabel("Frequency",fontsize=18)
ax1.set_xlim(0.0, 1.0)
ax1.set_ylim(0, 5)
ax1.legend()

sns.distplot(df_CPI['filled_circular_area_ratio'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI aggregates', ax=ax3)
sns.distplot(truncall['area ratio'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'#0B1CE0'}, label='IPAS $n_m$ $\leq$ 30', ax=ax3)
sns.distplot(trunc20['area ratio'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'--', 'color':'#0B1CE0'}, label='IPAS $n_m$ $\leq$ 20', ax=ax3)
sns.distplot(trunc10['area ratio'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle':'-.', 'color':'#0B1CE0'}, label='IPAS $n_m$ $\leq$ 10', ax=ax3)

ax3.set_xlabel("Area Ratio",fontsize=18)
ax3.set_ylabel("Frequency",fontsize=18)
ax3.set_xlim(0.0, 1.0)
ax3.set_ylim(0, 5)
ax3.legend()

ax4 = sns.boxplot(x="variable", y="Value", hue="Source", data=cdf, palette={'CPI': 'red', 'IPAS':'#0B1CE0'}, showfliers=False, ax=ax4)

for i in range(0,6,2):
    cpi_color = ax4.artists[i]
    cpi_color.set_facecolor('red')
for i in range(1,6,2):
    ipas_color = ax4.artists[i]
    ipas_color.set_facecolor('#0B1CE0')
ax4.set_xlabel("Bulk Statistics",fontsize=18)
#ax4.legend()

plt.savefig('../plots/CPI_IPAS_distributions.png', dpi=300, bbox_inches='tight')

In [ ]:
st.mode(truncall['area ratio']),st.mode(truncall['area ratio'])

In [ ]:
bins, edges = np.histogram(trunc5['cplx'], density=True, bins=70)
edges[np.where(bins == np.max(bins))]

CPI aggregates a little less complex 

CPI data looks more quasi-spherical in terms of aspect ratios: add in other particle types, not just aggs

# All CPI particle types

In [ ]:
#read in CPI data 
#all campaings in one file
df_CPI = pd.read_csv('final_databases/no_mask/all_campaigns.csv')
df_CPI = df_CPI[df_CPI['classification'] != 'blank']

In [ ]:
len(df_CPI), len(ddf), len(phi2D.flatten())

In [ ]:
#phi2D and phi2Ds from 0.25-10.0, 30 monomers, r_m = 10
#agg_phi, 3D from database from 0.25-10.0, 30 monomers, all r_m
fig, ax = plt.subplots(figsize=(7,5))

sns.distplot(phi2D[1:-1,:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='IPAS 2D')
sns.distplot(phi2Ds[1:-1,:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--', 'color': 'lightblue'}, label='IPAS mean 2D')
sns.distplot(ddf['agg_phi'][(ddf['mono_phi'] <= 10.0) & (ddf['mono_phi'] >= 0.25)], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color': 'orange'}, label='IPAS 3D')
sns.distplot(df_CPI['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI')
ax.set(xlabel='Aspect Ratio', ylabel='Frequency');

In [ ]:
n, bins, patches = plt.hist(df_CPI['complexity'], bins = 40)
bins[np.where(n == n.max())]  #max CPI aspect ratio from ellipse (not averaged)

In [ ]:
#mode of complexity histogram only dropped by 0.035 including all particle types

In [ ]:
#play around with specific particle types
df_CPI['classification'].unique()

In [ ]:
df_CPI = pd.read_csv('final_databases/no_mask/all_campaigns.csv')
df_CPI = df_CPI[(df_CPI['classification'] == 'agg') | \
               (df_CPI['classification'] == 'rimed agg') | \
               (df_CPI['classification'] == 'bullet') | \
               (df_CPI['classification'] == 'compact irregular')]

In [ ]:
#phi2D and phi2Ds from 0.25-10.0, 30 monomers, r_m = 10
#agg_phi, 3D from database from 0.25-10.0, 30 monomers, all r_m
fig, ax = plt.subplots(figsize=(7,5))

sns.distplot(phi2D[:,:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='IPAS 2D')
sns.distplot(phi2Ds[:,:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--', 'color': 'lightblue'}, label='IPAS mean 2D')
sns.distplot(ddf['agg_phi'][(ddf['mono_phi'] <= 10.0) & (ddf['mono_phi'] >= 0.25)], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color': 'orange'}, label='IPAS 3D')
sns.distplot(df_CPI['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI')
ax.set(xlabel='Aspect Ratio', ylabel='Frequency');

In [ ]:
df_CPI = pd.read_csv('final_databases/no_mask/all_campaigns.csv')

In [ ]:
df_CPI_bullet = df_CPI[(df_CPI['classification'] == 'bullet')]

In [ ]:
df_CPI_plate = df_CPI[(df_CPI['classification'] == 'plate')]

In [ ]:
df_CPI_agg = df_CPI[(df_CPI['classification'] == 'agg')]

In [ ]:
df_CPI_rimed_agg = df_CPI[(df_CPI['classification'] == 'rimed agg')]

In [ ]:
df_CPI_compact = df_CPI[(df_CPI['classification'] == 'compact irregular')]

In [ ]:
df_CPI_rimed_col = df_CPI[(df_CPI['classification'] == 'rimed column')]

In [ ]:
df_CPI_column = df_CPI[(df_CPI['classification'] == 'column') | (df_CPI['classification'] == 'needle')]

In [ ]:
#phi2D and phi2Ds from 0.25-10.0, 30 monomers, r_m = 10
#agg_phi, 3D from database from 0.25-10.0, 30 monomers, all r_m
fig, ax = plt.subplots(figsize=(7,5))

sns.distplot(phi2D[1:-1,:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='IPAS 2D')
sns.distplot(phi2Ds[1:-1,:,:,:].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--', 'color': 'lightblue'}, label='IPAS mean 2D')
sns.distplot(ddf['agg_phi'][(ddf['mono_phi'] <= 10.0) & (ddf['mono_phi'] >= 0.25)], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color': 'orange'}, label='IPAS 3D')
sns.distplot(df_CPI_rimed_agg['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI')
ax.set(xlabel='Aspect Ratio', ylabel='Frequency');

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
sns.distplot(trunc['cplx'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='IPAS')
sns.distplot(df_CPI_rimed_col['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI')
ax.set(xlabel='Complexity', ylabel='Frequency');

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
sns.distplot(trunc['cplx'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='IPAS')
sns.distplot(df_CPI_column['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'color':'red'}, label='CPI')
ax.set(xlabel='Complexity', ylabel='Frequency');

In [ ]:
#truncate # of monomers
trunc = ddf[(ddf['mono_phi'] > 0.01) & (ddf['mono_phi'] < 50) & (ddf['ncrystals'] < 20)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.distplot(df_CPI_agg['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='aggregate')

sns.distplot(df_CPI_bullet['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='bullet rosette')

sns.distplot(df_CPI_compact['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='compact irregular')

sns.distplot(df_CPI_column['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='column')

sns.distplot(df_CPI_plate['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='plate')

sns.distplot(df_CPI_rimed_agg['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='rimed agg')

sns.distplot(df_CPI_rimed_col['complexity'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='rimed column')

sns.distplot(trunc['cplx'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--'}, label='IPAS $n_m$ < 20')


ax.set(xlabel='Complexity', ylabel='Frequency');

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.distplot(df_CPI_agg['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'red'}, label='aggregate')

sns.distplot(phi2Ds[1:-1,:,:20].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--','color':'blue'}, label='IPAS avg 2D $n_m$ < 20')

sns.distplot(phi2Ds[1:-1,:,:10].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'blue'}, label='IPAS avg 2D $n_m$ < 10')

sns.distplot(phi2D[1:-1,:,:20].flatten(), hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--','color':'lightblue'}, label='IPAS 2D $n_m$ < 20')

sns.distplot(trunc['phi2D'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'lightblue'}, label='IPAS db 2D $n_m$ < 20')
#phi_m 0.01 and 50^
sns.distplot(trunc['agg_phi'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--','color':'orange'}, label='IPAS db 3D $n_m$ < 20')
#phi_m 0.01 and 50^
sns.distplot(ddf['agg_phi'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3,'color':'orange'}, label='IPAS db 3D $n_m$ < 30')


ax.set(xlabel='Aspect Ratio', ylabel='Frequency');

In [ ]:
trunc = ddf[(ddf['mono_phi'] > 0.01) & (ddf['mono_phi'] < 50) & (ddf['ncrystals'] < 10)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
#trunc = ddf[(ddf['mono_phi'] > 0.01) & (ddf['mono_phi'] < 50) & (ddf['ncrystals'] < 30)]
sns.distplot(df_CPI_bullet['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='bullet rosette')

sns.distplot(df_CPI_compact['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='compact irregular')

sns.distplot(df_CPI_column['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='column')

sns.distplot(df_CPI_plate['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='plate')

sns.distplot(df_CPI_rimed_agg['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='rimed aggregate')

sns.distplot(df_CPI_rimed_col['phi_ellipse'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3}, label='rimed column')

sns.distplot(ddf['agg_phi'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--'}, label='IPAS 3D $n_m$ < 20')

sns.distplot(ddf['phi2D'], hist = False, kde = True, norm_hist=True,
             kde_kws = {'linewidth': 3, 'linestyle': '--'}, label='IPAS 2D $n_m$ < 20')
